In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import polars as pl
import os
import pyarrow.parquet as pq

In [2]:
def find_columns_in_parquets(directory, columns):
    # List all parquet files in the directory
    parquet_files = [f for f in os.listdir(directory) if f.endswith('.parquet')]
    
    # Dictionary to keep track of which files contain each column
    column_presence = {column: [] for column in columns}
    
    # Check each file
    for file in parquet_files:
        file_path = os.path.join(directory, file)
        # Read the schema of the parquet file without loading the data
        parquet_file = pq.ParquetFile(file_path)
        
        # Get the names of the columns in the current parquet file
        file_columns = parquet_file.schema.names
        
        # Check for each target column if it is in the current file's columns
        for column in columns:
            if column in file_columns:
                column_presence[column].append(file)

    return column_presence

# Directory containing the Parquet files
directory = '../public/home-credit-credit-risk-model-stability/parquet_files/train'

# List of columns you are looking for
columns = [
    "actualdpd_943P", "purposeofcred_426M", "amount_1115A", "credacc_actualbalance_314A",
    "actualdpdtolerance_344P", "annuity_780A", "numinstpaidearly_338L", "empl_employedtotal_800L",
    "empl_industry_691L", "maininc_215A", "debtoverdue_47A", "totalsettled_863A",
    "totaloutstanddebtvalue_39A", "avgdbddpdlast24m_3658932P", "avgdbddpdlast3m_4187120P",
    "clientscnt12m_3712952L", "applicationscnt_1086L", "applicationcnt_361L", "applications30d_658L"
]

# Find columns
column_presence = find_columns_in_parquets(directory, columns)

# Print out results
for column, files in column_presence.items():
    print(f"Column '{column}' is found in files: {files}")


Column 'actualdpd_943P' is found in files: ['train_applprev_1_1.parquet', 'train_applprev_1_0.parquet']
Column 'purposeofcred_426M' is found in files: ['train_credit_bureau_a_1_0.parquet', 'train_credit_bureau_a_1_1.parquet', 'train_credit_bureau_a_1_3.parquet', 'train_credit_bureau_a_1_2.parquet']
Column 'amount_1115A' is found in files: ['train_credit_bureau_b_1.parquet']
Column 'credacc_actualbalance_314A' is found in files: ['train_applprev_1_1.parquet', 'train_applprev_1_0.parquet']
Column 'actualdpdtolerance_344P' is found in files: ['train_static_0_1.parquet', 'train_static_0_0.parquet']
Column 'annuity_780A' is found in files: ['train_static_0_1.parquet', 'train_static_0_0.parquet']
Column 'numinstpaidearly_338L' is found in files: ['train_static_0_1.parquet', 'train_static_0_0.parquet']
Column 'empl_employedtotal_800L' is found in files: ['train_person_1.parquet']
Column 'empl_industry_691L' is found in files: ['train_person_1.parquet']
Column 'maininc_215A' is found in files:

In [3]:
# Function to read specific columns from a Parquet file
def read_columns(file_path, columns):
    return pd.read_parquet(file_path, columns=columns)

# Base DataFrame from 'train_base' with 'case_id' and 'date_decision'
base_directory = '../public/home-credit-credit-risk-model-stability/parquet_files/train/'
base_df = read_columns(base_directory + 'train_base.parquet', ['case_id', 'date_decision', 'target'])

# Dictionary of file groups and their specific columns to join with base_df
file_groups = {
    'train_applprev': {
        'files': ['train_applprev_1_1.parquet', 'train_applprev_1_0.parquet'],
        'columns': ['case_id', 'actualdpd_943P']
    },
    'train_credit_bureau_a': {
        'files': [
            'train_credit_bureau_a_1_0.parquet', 'train_credit_bureau_a_1_1.parquet',
            'train_credit_bureau_a_1_2.parquet', 'train_credit_bureau_a_1_3.parquet'
        ],
        'columns': ['case_id', 'purposeofcred_426M', 'debtoverdue_47A', 'totaloutstanddebtvalue_39A']
    },
    'train_credit_bureau_b': {
        'files': ['train_credit_bureau_b_1.parquet'],
        'columns': ['case_id', 'amount_1115A']
    },
    'train_static': {
        'files': ['train_static_0_1.parquet', 'train_static_0_0.parquet'],
        'columns': ['case_id', 'actualdpdtolerance_344P', 'annuity_780A', 'numinstpaidearly_338L', 'maininc_215A', 'totalsettled_863A', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'clientscnt12m_3712952L', 'applicationscnt_1086L', 'applicationcnt_361L', 'applications30d_658L']
    },
    'train_person': {
        'files': ['train_person_1.parquet'],
        'columns': ['case_id', 'empl_employedtotal_800L', 'empl_industry_691L']
    }
}

# Concatenate files within the same group and join with the base dataframe
for group, details in file_groups.items():
    group_df = pd.concat([read_columns(base_directory + file, details['columns']) for file in details['files']], ignore_index=True)
    base_df = base_df.merge(group_df.drop_duplicates(subset='case_id'), on='case_id', how='left')

# Show the combined dataframe
base_df.head()


,case_id,date_decision,target,actualdpd_943P,purposeofcred_426M,debtoverdue_47A,totaloutstanddebtvalue_39A,amount_1115A,actualdpdtolerance_344P,annuity_780A,...,maininc_215A,totalsettled_863A,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,clientscnt12m_3712952L,applicationscnt_1086L,applicationcnt_361L,applications30d_658L,empl_employedtotal_800L,empl_industry_691L
0,0,2019-01-03,0,NaN,NaN,NaN,NaN,NaN,NaN,1917.6,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,MORE_FIVE,OTHER
1,1,2019-01-03,0,NaN,NaN,NaN,NaN,NaN,NaN,3134.0,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,MORE_FIVE,OTHER
2,2,2019-01-04,0,0.0,NaN,NaN,NaN,NaN,NaN,4937.0,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,MORE_FIVE,OTHER
3,3,2019-01-03,0,0.0,NaN,NaN,NaN,NaN,NaN,4643.6,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0,MORE_FIVE,OTHER
4,4,2019-01-04,1,0.0,NaN,NaN,NaN,NaN,NaN,3390.2,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0,MORE_FIVE,OTHER


In [4]:
# Count the number of NaN values in each column
nan_counts = base_df.isna().sum()

# Calculate the percentage of NaN values in each column
nan_percentage = (nan_counts / len(base_df)) * 100

# Combine the counts and percentages into a DataFrame for better visualization
nan_distribution = pd.DataFrame({
    'Number of NaNs': nan_counts,
    'Percentage of NaNs': nan_percentage
})

# Print the distribution of NaN values
nan_distribution


,Number of NaNs,Percentage of NaNs
case_id,0,0.000000
date_decision,0,0.000000
target,0,0.000000
actualdpd_943P,305154,19.988354
purposeofcred_426M,140386,9.195636
debtoverdue_47A,756013,49.520751
totaloutstanddebtvalue_39A,834440,54.657916
amount_1115A,1501222,98.333813
actualdpdtolerance_344P,418178,27.391710
annuity_780A,0,0.000000


In [5]:
# Assuming base_df is already loaded

# List of all columns to impute with zero
zero_impute_columns = [
    'actualdpd_943P',
    'numinstpaidearly_338L',
    'debtoverdue_47A',
    'totalsettled_863A',
    'clientscnt12m_3712952L',
    'applicationscnt_1086L',
    'applicationcnt_361L',
    'applications30d_658L',
    'totaloutstanddebtvalue_39A',
    'maininc_215A',
    'avgdbddpdlast3m_4187120P',
    'avgdbddpdlast24m_3658932P',
    'actualdpdtolerance_344P'
]

# Impute NaN values with 0 for each column in the list
for column in zero_impute_columns:
    base_df[column].fillna(0, inplace=True)

# Drop specific columns
columns_to_drop = [
    'empl_employedtotal_800L',
    'empl_industry_691L',
    'amount_1115A'  # Adding 'amount_1115A' to the list of columns to drop
]

base_df.drop(columns=columns_to_drop, inplace=True)

# Replace NaN values with the mode in 'purposeofcred_426M'
mode_value = base_df['purposeofcred_426M'].mode()[0]  # Get the mode of the column
base_df['purposeofcred_426M'].fillna(mode_value, inplace=True)

/tmp/ipykernel_2353/2441390960.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base_df[column].fillna(0, inplace=True)
/tmp/ipykernel_2353/2441390960.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [6]:
base_df.head()  

,case_id,date_decision,target,actualdpd_943P,purposeofcred_426M,debtoverdue_47A,totaloutstanddebtvalue_39A,actualdpdtolerance_344P,annuity_780A,numinstpaidearly_338L,maininc_215A,totalsettled_863A,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,clientscnt12m_3712952L,applicationscnt_1086L,applicationcnt_361L,applications30d_658L
0,0,2019-01-03,0,0.0,60c73645,0.0,0.0,0.0,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-01-03,0,0.0,60c73645,0.0,0.0,0.0,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2019-01-04,0,0.0,60c73645,0.0,0.0,0.0,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2019-01-03,0,0.0,60c73645,0.0,0.0,0.0,4643.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,2019-01-04,1,0.0,60c73645,0.0,0.0,0.0,3390.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
# Count the number of NaN values in each column
nan_counts = base_df.isna().sum()

# Calculate the percentage of NaN values in each column
nan_percentage = (nan_counts / len(base_df)) * 100

# Combine the counts and percentages into a DataFrame for better visualization
nan_distribution = pd.DataFrame({
    'Number of NaNs': nan_counts,
    'Percentage of NaNs': nan_percentage
})

# Print the distribution of NaN values
nan_distribution

,Number of NaNs,Percentage of NaNs
case_id,0,0.0
date_decision,0,0.0
target,0,0.0
actualdpd_943P,0,0.0
purposeofcred_426M,0,0.0
debtoverdue_47A,0,0.0
totaloutstanddebtvalue_39A,0,0.0
actualdpdtolerance_344P,0,0.0
annuity_780A,0,0.0
numinstpaidearly_338L,0,0.0
